In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)
data = {
    'plots': {},
    'stats': {}
}

In [ ]:
confirmed = pd.read_csv('datasets/confirmed_transposed.csv', index_col=0)
deaths = pd.read_csv('datasets/deaths_transposed.csv', index_col=0)
recovered = pd.read_csv('datasets/recovered_transposed.csv', index_col=0)

In [ ]:
colors = ['rgb(255,0,0)', 'rgb(0,230,0)', 'rgb(0,50,255)', 'rgb(255,150,0)', 'rgb(0,150,255)', 'rgb(150,0,255)']
def get_color(i, limit):
    if (i >= len(colors) or i >= limit):
        return 'rgb(150,150,150)', False
    return colors[i], True

In [ ]:
def sort_columns_on_last_row(df):
    sorted_indicies = (-df.loc[df.last_valid_index()]).argsort()
    return df[df.columns[sorted_indicies]]

In [ ]:
def over_threshold(df, threshold):
    mask = (df.iloc[-1] > threshold).index[df.iloc[-1] > threshold]
    over_threshold = df[mask]
    return sort_columns_on_last_row(over_threshold)

In [ ]:
threshold = 500
confirmed_over_threshold = over_threshold(confirmed, threshold)

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(
    layout=go.Layout(
        title=go.layout.Title(text=f'Countries with over {threshold} confirmed cases due to COVID-19'),
        plot_bgcolor='rgb(255,255,255)',
        yaxis=go.layout.YAxis(
            showgrid=True,
            automargin=True,
            gridwidth=1,
            gridcolor='rgb(200,200,200)'
        ),
        xaxis=go.layout.XAxis(
            showgrid=True,
            automargin=True,
            gridwidth=1,
            gridcolor='rgb(200,200,200)',
            dtick=4
        ),
        legend_orientation="h",
    )
)

for i, column in enumerate(confirmed_over_threshold):
    color, _ = get_color(i, 6)
    fig.add_trace(go.Scatter(
        x=confirmed_over_threshold[column].index, 
        y=confirmed_over_threshold[column].values,
        name=column,
        marker_color=color,
        line=dict(width=1)
    ))
fig.show()
data['plots']['figure_1'] = fig.to_json()

In [ ]:
def get_from_first_occurrence(df, country, above_value):
    above = df[country][df[country] > above_value]
    return np.arange(0, len(above)), above.values

In [ ]:
threshold = 5
deaths_over_threshold = over_threshold(deaths, threshold)
deaths_china = deaths_over_threshold['China']
deaths_over_threshold = deaths_over_threshold.drop(['China'], axis=1)

fig = go.Figure(
    layout=go.Layout(
        title=go.layout.Title(text=f'Countries with over {threshold} deaths, aligned with second registered death'),
        plot_bgcolor='rgb(255,255,255)',
        yaxis=go.layout.YAxis(
            type='log', showgrid=True, gridwidth=1, gridcolor='rgb(200,200,200)'
        ),
        xaxis=go.layout.XAxis(showgrid=True, gridwidth=1, gridcolor='rgb(200,200,200)', dtick=4),
    )
)


fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction='left',
            buttons=[
                dict(
                    label="Logarithmic",
                    method="relayout",
                    args=["yaxis.type", 'log']
                ),
                dict(
                    label="Linear",
                    method="relayout",
                    args=["yaxis.type", 'lin']
                ),
            ],
            pad={"r": 0, "t": 0},
            showactive=True,
            x=0,
            xanchor="left",
            y=-0.1,
            yanchor="top"
        )
    ]
)

max_count = 0
for i, column in enumerate(deaths_over_threshold):
    x, y = get_from_first_occurrence(deaths_over_threshold, column, 1)
    if len(x) > max_count:
        max_count = len(x)
    color, show = get_color(i, 6)
    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        name=column,
        mode='lines',
        marker_color=color,
        line=dict(width=1, dash='solid' if show else 'dash'),
        visible=True if show else 'legendonly'
    ))

fig.add_trace(go.Scatter(
    x=np.arange(0, max_count), 
    y=np.array([2, 2, 4, 7, 11] + deaths_china.values.tolist())[:max_count],
    name='China',
    mode='lines',
    marker_color='rgb(255,0,150)',
    line=dict(width=1)
))
fig.show()
data['plots']['figure_2'] = fig.to_json()

In [ ]:
over_5_deaths = over_threshold(deaths, 5)
change = over_5_deaths.iloc[-1] - over_5_deaths.iloc[-2]

In [ ]:
data['stats']['change_deaths'] = change.to_json()

In [ ]:
data['stats']['num_countries_over_5_deaths'] = change.to_json()

In [ ]:
import json
with open('../../frontend/src/data/data.json', 'w') as f:
    json.dump(data, f)